<a href="https://colab.research.google.com/github/AUT-Student/CN-Project/blob/main/CN_Project_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [23]:
import torch
import scipy
import scipy.sparse as sp

In [7]:
!pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install torch-geometric
!pip install torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-1.13.0+cu116.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 44.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 KB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 31.1 MB/s eta 0:00:00
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773302 sha256=1697308f7c844e93d987487d524a43b37214f1a77b24380e3cc4a05502694175
  Stored in directory: /root/.cache/pip/wheels/59/a3/20/198928106d3169865ae73afcbd3d3d1796cf6b429b55c65378
Successfully built 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 873.9/873.9 KB 33.6 MB/s eta 0:00:00


Source: https://docs.dgl.ai/en/0.8.x/_modules/dgl/nn/pytorch/conv/sgconv.html

# Dataset

In [8]:
from torch_geometric.datasets.planetoid import Planetoid

In [9]:
planetoid = Planetoid("/content/planetoid", name="PubMed")

Processing...
Done!


In [10]:
dataset = planetoid[0]

In [11]:
dataset

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])

In [13]:
(sum(dataset.train_mask), sum(dataset.val_mask), sum(dataset.test_mask))

(tensor(60), tensor(500), tensor(1000))

# Create SGC Matrix

In [14]:
from torch_geometric.utils import to_scipy_sparse_matrix

In [16]:
A = to_scipy_sparse_matrix(dataset.edge_index)
I = sp.eye(dataset.num_nodes)

In [22]:
A_tilde = A + I

In [49]:
D_tilde = sp.diags(A_tilde.sum(axis=0).A1)

In [53]:
S = D_tilde.power(-1/2) * A * D_tilde.power(-1/2)

In [54]:
k=3

In [55]:
Sk = S.power(k)

In [66]:
X = sp.csr_matrix(dataset.x)

In [67]:
SkX = Sk * sp.csr_matrix(dataset.x)